In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import gdown
import os

In [2]:
file_id = '1BM7QmO4GoR5YxlMPfM5DLyrxBm3mn3oB'
dataset_link = f'https://drive.google.com/uc?id={file_id}'
output_file = 'after_eda_dataset.zip'

In [3]:
if not os.path.exists(output_file):
    gdown.download(dataset_link, output_file, quiet=False)
    print(f"Файл {output_file} скачан.")
else:
    print(f"Файл {output_file} уже существует.")

Файл after_eda_dataset.zip уже существует.


In [4]:
df = pd.read_csv(output_file, compression='zip')

Формулируем гипотезы:

Нулевая гипотеза (H0): Игра с сильнейшим противником не влияет на общее количество ошибок и неточностей.

Альтернативная гипотеза (HA): Игра с сильнейшим противником увеличивает общее количество ошибок и неточностей.

In [5]:
# Этап 1: Создание общего числа ошибок для обоих игроков
df['Total_Errors_WHITE'] = df['White_Inaccuracy'] + df['White_Mistake'] + df['White_Blunder']
df['Total_Errors_BLACK'] = df['Black_Inaccuracy'] + df['Black_Mistake'] + df['Black_Blunder']

# Этап 2: Определение сильных соперников (предположим порог различия в рейтинге 325)
rating_difference_threshold = 325

# Вычисляем разность рейтинга (белый игрок минус черный игрок)
df['Rating_Difference'] = abs(df['White_Rating'] - df['Black_Rating'])

# Группируем матчи на основании разницы в рейтинге
strong_opponent_games = df[df['Rating_Difference'] >= rating_difference_threshold]
weak_or_average_opponent_games = df[df['Rating_Difference'] < rating_difference_threshold]

# Этап 3: Извлекаем суммарные ошибки для обеих групп (отдельно для черных и белых)
total_errors_strong_white = strong_opponent_games['Total_Errors_WHITE'].values
total_errors_strong_black = strong_opponent_games['Total_Errors_BLACK'].values

total_errors_weak_white = weak_or_average_opponent_games['Total_Errors_WHITE'].values
total_errors_weak_black = weak_or_average_opponent_games['Total_Errors_BLACK'].values

# Этап 4: Применение t-теста для оценки различий между группами
# Проверка изменения качества игры белого игрока
statistic_white, p_value_white = ttest_ind(total_errors_strong_white, total_errors_weak_white)
print(f"Тестовое значение (белый игрок): {statistic_white}, P-value: {p_value_white}")

# Анализ результатов
if p_value_white < 0.05:
    print("Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.")
else:
    print("Гипотеза принимается: нет доказательств существенного влияния силы противника на количество ошибок.")
    
# Проверка изменения качества игры черного игрока
statistic_black, p_value_black = ttest_ind(total_errors_strong_black, total_errors_weak_black)
print(f"Тестовое значение (черный игрок): {statistic_black}, P-value: {p_value_black}")

# Анализ результатов
if p_value_black < 0.05:
    print("Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.")
else:
    print("Гипотеза принимается: нет доказательств существенного влияния силы противника на количество ошибок.")

Тестовое значение (белый игрок): -2.1614931567646543, P-value: 0.030659367732580007
Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.
Тестовое значение (черный игрок): -5.860439247408993, P-value: 4.629258892176753e-09
Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.


ВЫВОД:

Используем подходящий статистический критерий для сравнения средних значений двух групп (игр с сильными игроками и игр со средними/более слабыми). Наиболее подходящим методом здесь является t-тест, поскольку мы сравниваем средние значения между двумя независимыми выборками.

По нашему тесту видно, что существует прямая зависимость между силой игрока и количеством допускаемых ошибок при игре с ним. НО эта зависимость есть, если разница в рейтинге более 325. Этот порог выявили экспериментальным путем подбора разницы рейтингов, так как если рейтинг ниже 325, то у белых игроков такой зависимости уже не наблюдается.